In [1]:
import os
os.chdir( "../")

In [2]:
import pandas as pd
import numpy as np

path = "Dataset/data_train.csv"
df = pd.read_csv(filepath_or_buffer=path,
                               sep=",",
                               header=1,
                               engine='python',
                               names=['UserID', 'ItemID', 'Interaction'])


df

,UserID,ItemID,Interaction
0,1,15,1.0
1,1,16,1.0
2,1,133,1.0
3,1,161,1.0
4,1,187,1.0
...,...,...,...
478724,13024,13605,1.0
478725,13024,13823,1.0
478726,13024,15122,1.0
478727,13024,18185,1.0


In [3]:
df.Interaction.value_counts()

1.0    478729
Name: Interaction, dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478729 entries, 0 to 478728
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   UserID       478729 non-null  int64  
 1   ItemID       478729 non-null  int64  
 2   Interaction  478729 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 11.0 MB


In [5]:
user_ids = df["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
item_ids = df["ItemID"].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
df["User"] = df["UserID"].map(user2user_encoded)
df["Item"] = df["ItemID"].map(item2item_encoded)

num_users = len(user2user_encoded)
num_items = len(item_encoded2item)
df["Interaction"] = df["Interaction"].values.astype(np.float32)

# min and max ratings will be used to normalize the ratings later
min_rating = 0.0
max_rating = max(df["Interaction"])

print(
    "Number of users: {}, Number of Items: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_items, min_rating, max_rating
    )
)

Number of users: 12638, Number of Items: 22222, Min rating: 0.0, Max rating: 1.0


In [6]:
userId_unique = df["UserID"].unique()
itemId_unique = df["ItemID"].unique()

In [7]:
from sklearn.model_selection import train_test_split
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample



urm_all = sps.coo_matrix((df["Interaction"].values, 
                          (df["User"].values, df["Item"].values)))

urm_train_validation, urm_test = split_train_in_two_percentage_global_sample(urm_all, train_percentage = 0.80)
urm_train, urm_validation = split_train_in_two_percentage_global_sample(urm_train_validation, train_percentage = 0.80)

In [8]:
num_users = len(userId_unique)
num_items = len(itemId_unique)

In [9]:
from Recommenders.Recommender_import_list import *
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10], ignore_users=[])

/Users/japo/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


EvaluatorHoldout: Ignoring 2602 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [10]:
import implicit
from Recommenders.MatrixFactorization.ImplicitIALSRecommenderWrap import ImplicitIALSRecommenderWrap
#imp = implicit.als.AlternatingLeastSquares(factors=50,iterations=10)

#imp.fit(urm_train)

#mod = ImplicitIALSRecommenderWrap(imp, urm_train)

#users_to_evaluate = np.arange(urm_validation.shape[0])
#mod.recommend(users_to_evaluate)

In [11]:
#scores = mod._compute_item_score(users_to_evaluate)
#scores.shape
#scores[0]

In [12]:

#imp.fit(urm_train)

#mod = ImplicitIALSRecommenderWrap(imp,urm_train)
#evaluator_validation.evaluateRecommender(mod)

In [13]:
#scores = mod._compute_item_score(users_to_evaluate)
#scores.shape
#scores[0]

In [1]:
import implicit
ials = implicit.als.AlternatingLeastSquares(factors=10, regularization=0.01, iterations=10)

ials.fit(urm_train,True)
ials.recommend(0,urm_train[0],N=10)
ials.fit(urm_train,True)
ials.recommend(0,urm_train[0],N=10)
ials.fit(urm_train,True)
ials.recommend(0,urm_train[0],N=10)


NameError: name 'implicit' is not defined

In [14]:
from Recommenders.MatrixFactorization.ImplicitIALSRecommender import ImplicitIALSRecommender

metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

earlystopping_keywargs = {"validation_every_n": 1,
                          "stop_on_validation": True,
                          "evaluator_object": evaluator_validation,
                          "lower_validations_allowed": 5,
                          "validation_metric": metric_to_optimize,
                          }

implicit_model = ImplicitIALSRecommender(urm_train,factors=50)
implicit_model.fit(epochs=100,**earlystopping_keywargs)

  0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 25/25 [00:45<00:00,  1.84s/it]


EvaluatorHoldout: Processed 10036 (100.0%) in 14.85 sec. Users per second: 676


  0%|          | 0/25 [00:00<?, ?it/s]

Current MAP: 0.0029178449644771086 @ epoch : 1
Curr scores:[[ 3.62600952e-01  2.05647260e-01  1.15094535e-01 ... -3.40282347e+38
  -3.40282347e+38 -3.40282347e+38]]
New best model found! MAP: 0.0029178449644771086 @ epoch : 1


...

EvaluatorHoldout: Processed 10036 (100.0%) in 15.32 sec. Users per second: 655


  0%|          | 0/25 [00:00<?, ?it/s]

Current MAP: 0.0029178449644771086 @ epoch : 5
Curr scores:[[ 3.31260949e-01  1.90454125e-01  1.15963444e-01 ... -3.40282347e+38
  -3.40282347e+38 -3.40282347e+38]]


100%|██████████| 25/25 [00:49<00:00,  1.96s/it]


EvaluatorHoldout: Processed 10036 (100.0%) in 15.48 sec. Users per second: 648
Current MAP: 0.0029178449644771086 @ epoch : 6
Curr scores:[[ 3.2576874e-01  1.9081752e-01  1.1497831e-01 ... -3.4028235e+38
  -3.4028235e+38 -3.4028235e+38]]
Implicit-IALSRecommender: Convergence reached! Terminating at epoch 6. Best value for 'MAP' at epoch 1 is 0.0029


In [ ]:
len(implicit_model.recommend(1,return_scores=True)[0])

PARAMS: user_id_array=1, remove_seen_flag=True, items_to_compute=None, remove_top_pop_flag=False, remove_custom_items_flag=False, return_scores=True


22200

In [ ]:
np.array(implicit_model.recommend([0,1],return_scores=True)[1]).shape

PARAMS: user_id_array=[0, 1], remove_seen_flag=True, items_to_compute=None, remove_top_pop_flag=False, remove_custom_items_flag=False, return_scores=True


(2, 22168)

In [2]:
implicit_model = ImplicitIALSRecommender(urm_train,50)
implicit_model.fit(epochs=100,**earlystopping_keywargs)

NameError: name 'ImplicitIALSRecommender' is not defined